In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 5)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 3.2604368925094604, Final Batch Loss: 1.0923066139221191
Epoch 2, Loss: 3.247307777404785, Final Batch Loss: 1.0743658542633057
Epoch 3, Loss: 3.244359254837036, Final Batch Loss: 1.0873245000839233
Epoch 4, Loss: 3.244392156600952, Final Batch Loss: 1.0947113037109375
Epoch 5, Loss: 3.231306314468384, Final Batch Loss: 1.079217791557312
Epoch 6, Loss: 3.224176526069641, Final Batch Loss: 1.0788626670837402
Epoch 7, Loss: 3.212392568588257, Final Batch Loss: 1.075840950012207
Epoch 8, Loss: 3.1904834508895874, Final Batch Loss: 1.048733115196228
Epoch 9, Loss: 3.1881483793258667, Final Batch Loss: 1.0706324577331543
Epoch 10, Loss: 3.1638439893722534, Final Batch Loss: 1.0475085973739624
Epoch 11, Loss: 3.1571143865585327, Final Batch Loss: 1.0665897130966187
Epoch 12, Loss: 3.119445562362671, Final Batch Loss: 1.039324164390564
Epoch 13, Loss: 3.0842329263687134, Final Batch Loss: 1.0190354585647583
Epoch 14, Loss: 3.0516557693481445, Final Batch Loss: 1.015592336654663

Epoch 112, Loss: 0.3059864416718483, Final Batch Loss: 0.09663908183574677
Epoch 113, Loss: 0.25133049860596657, Final Batch Loss: 0.07476981729269028
Epoch 114, Loss: 0.22448139637708664, Final Batch Loss: 0.09522062540054321
Epoch 115, Loss: 0.21518034860491753, Final Batch Loss: 0.03550456091761589
Epoch 116, Loss: 0.1985253021121025, Final Batch Loss: 0.030714109539985657
Epoch 117, Loss: 0.25120290368795395, Final Batch Loss: 0.05948729068040848
Epoch 118, Loss: 0.20078280195593834, Final Batch Loss: 0.0382009781897068
Epoch 119, Loss: 0.274280846118927, Final Batch Loss: 0.1045471578836441
Epoch 120, Loss: 0.24227990582585335, Final Batch Loss: 0.10553690791130066
Epoch 121, Loss: 0.2517029158771038, Final Batch Loss: 0.05814242735505104
Epoch 122, Loss: 0.21462027728557587, Final Batch Loss: 0.06380203366279602
Epoch 123, Loss: 0.2476310357451439, Final Batch Loss: 0.11554593592882156
Epoch 124, Loss: 0.22515712678432465, Final Batch Loss: 0.06796742975711823
Epoch 125, Loss: 0.

Epoch 226, Loss: 0.16384362615644932, Final Batch Loss: 0.060432318598032
Epoch 227, Loss: 0.1456424668431282, Final Batch Loss: 0.043471530079841614
Epoch 228, Loss: 0.12595934979617596, Final Batch Loss: 0.0276006031781435
Epoch 229, Loss: 0.1649528332054615, Final Batch Loss: 0.07057897746562958
Epoch 230, Loss: 0.170269887894392, Final Batch Loss: 0.05601052939891815
Epoch 231, Loss: 0.13846344873309135, Final Batch Loss: 0.03289224952459335
Epoch 232, Loss: 0.15925528667867184, Final Batch Loss: 0.06524311006069183
Epoch 233, Loss: 0.17631692253053188, Final Batch Loss: 0.09764669835567474
Epoch 234, Loss: 0.12454153224825859, Final Batch Loss: 0.014943383634090424
Epoch 235, Loss: 0.15399792417883873, Final Batch Loss: 0.036055441945791245
Epoch 236, Loss: 0.10579955019056797, Final Batch Loss: 0.04582373425364494
Epoch 237, Loss: 0.11644266545772552, Final Batch Loss: 0.027970753610134125
Epoch 238, Loss: 0.16486212238669395, Final Batch Loss: 0.08295343816280365
Epoch 239, Loss

Epoch 339, Loss: 0.12507102638483047, Final Batch Loss: 0.019910115748643875
Epoch 340, Loss: 0.07847681641578674, Final Batch Loss: 0.013336708769202232
Epoch 341, Loss: 0.10658838972449303, Final Batch Loss: 0.014708191156387329
Epoch 342, Loss: 0.15185553207993507, Final Batch Loss: 0.08283641934394836
Epoch 343, Loss: 0.08011816069483757, Final Batch Loss: 0.019245581701397896
Epoch 344, Loss: 0.12524322140961885, Final Batch Loss: 0.040476951748132706
Epoch 345, Loss: 0.14335806854069233, Final Batch Loss: 0.07770899683237076
Epoch 346, Loss: 0.1516022626310587, Final Batch Loss: 0.1064026728272438
Epoch 347, Loss: 0.14036710560321808, Final Batch Loss: 0.05265795812010765
Epoch 348, Loss: 0.1244139987975359, Final Batch Loss: 0.05804908275604248
Epoch 349, Loss: 0.1442355653271079, Final Batch Loss: 0.0797082856297493
Epoch 350, Loss: 0.09420447424054146, Final Batch Loss: 0.03353453055024147
Epoch 351, Loss: 0.12120024673640728, Final Batch Loss: 0.06677079945802689
Epoch 352, L

Epoch 453, Loss: 0.09411703236401081, Final Batch Loss: 0.02697077952325344
Epoch 454, Loss: 0.07861884776502848, Final Batch Loss: 0.009278235025703907
Epoch 455, Loss: 0.0821051336824894, Final Batch Loss: 0.015054140239953995
Epoch 456, Loss: 0.08794862870126963, Final Batch Loss: 0.009803486056625843
Epoch 457, Loss: 0.07733837235718966, Final Batch Loss: 0.030808182433247566
Epoch 458, Loss: 0.0805782787501812, Final Batch Loss: 0.027099916711449623
Epoch 459, Loss: 0.06489600567147136, Final Batch Loss: 0.004706739913672209
Epoch 460, Loss: 0.08211303502321243, Final Batch Loss: 0.04224906116724014
Epoch 461, Loss: 0.08703353255987167, Final Batch Loss: 0.034636929631233215
Epoch 462, Loss: 0.07503071054816246, Final Batch Loss: 0.016214150935411453
Epoch 463, Loss: 0.0987707618623972, Final Batch Loss: 0.04480660706758499
Epoch 464, Loss: 0.07401635404676199, Final Batch Loss: 0.012677990831434727
Epoch 465, Loss: 0.04896857123821974, Final Batch Loss: 0.00889732874929905
Epoch 

...

Epoch 4533, Loss: 0.00022207465008250438, Final Batch Loss: 0.00010565969569142908
Epoch 4534, Loss: 0.0007380193419521675, Final Batch Loss: 3.6708654079120606e-05
Epoch 4535, Loss: 0.0004266225514584221, Final Batch Loss: 4.3440690205898136e-05
Epoch 4536, Loss: 0.00031990319985197857, Final Batch Loss: 1.7906066204886883e-05
Epoch 4537, Loss: 0.0009743516711750999, Final Batch Loss: 0.0007807968650013208
Epoch 4538, Loss: 0.0004292314115446061, Final Batch Loss: 0.0002707074163481593
Epoch 4539, Loss: 0.002564633196016075, Final Batch Loss: 0.0001405584189342335
Epoch 4540, Loss: 0.00019903449992852984, Final Batch Loss: 9.660237083153334e-06
Epoch 4541, Loss: 0.00010228564769931836, Final Batch Loss: 1.3521402252081316e-05
Epoch 4542, Loss: 0.00012989104106964078, Final Batch Loss: 1.2911088560940698e-05
Epoch 4543, Loss: 0.0004336184174462687, Final Batch Loss: 1.5605823136866093e-05
Epoch 4544, Loss: 0.0002208787827839842, Final Batch Loss: 0.00014148687478154898
Epoch 4545, Loss

Epoch 4642, Loss: 0.00023696569587627891, Final Batch Loss: 2.880813008232508e-05
Epoch 4643, Loss: 0.0034969841926795198, Final Batch Loss: 2.5597631974960677e-05
Epoch 4644, Loss: 0.00035782952545559965, Final Batch Loss: 9.796478843782097e-05
Epoch 4645, Loss: 0.00023320420223171823, Final Batch Loss: 3.57900389644783e-05
Epoch 4646, Loss: 0.00010815308905876009, Final Batch Loss: 4.80656890431419e-05
Epoch 4647, Loss: 0.00012210959721414838, Final Batch Loss: 1.1572201401577331e-05
Epoch 4648, Loss: 0.0003988535281678196, Final Batch Loss: 5.122009679325856e-05
Epoch 4649, Loss: 0.00042849469537031837, Final Batch Loss: 4.04546590289101e-05
Epoch 4650, Loss: 0.0011787027106038295, Final Batch Loss: 5.2784096624236554e-05
Epoch 4651, Loss: 0.007796736008458538, Final Batch Loss: 2.3844866518629715e-05
Epoch 4652, Loss: 0.0014070724428165704, Final Batch Loss: 0.0007505555404350162
Epoch 4653, Loss: 0.00020239160176060977, Final Batch Loss: 0.00011282923514954746
Epoch 4654, Loss: 0.

Epoch 4744, Loss: 0.00043045712300227024, Final Batch Loss: 4.57569258287549e-05
Epoch 4745, Loss: 0.023404930798278656, Final Batch Loss: 5.3671210480388254e-05
Epoch 4746, Loss: 0.008756675277254544, Final Batch Loss: 6.188363477122039e-05
Epoch 4747, Loss: 0.006645425237365998, Final Batch Loss: 0.00020253362890798599
Epoch 4748, Loss: 0.0009662016746005975, Final Batch Loss: 0.0006979848258197308
Epoch 4749, Loss: 0.003743929155461956, Final Batch Loss: 0.0030883417930454016
Epoch 4750, Loss: 0.0033366773241141345, Final Batch Loss: 0.002771733794361353
Epoch 4751, Loss: 0.000299456313769042, Final Batch Loss: 1.1256054676778149e-05
Epoch 4752, Loss: 0.01601397826016182, Final Batch Loss: 0.00011944053403567523
Epoch 4753, Loss: 0.00037432750468724407, Final Batch Loss: 8.266131044365466e-05
Epoch 4754, Loss: 0.002611540963698644, Final Batch Loss: 8.61323278513737e-05
Epoch 4755, Loss: 0.002502877761799027, Final Batch Loss: 1.625580080144573e-05
Epoch 4756, Loss: 0.00023477593640

Epoch 4856, Loss: 0.0015400590928038582, Final Batch Loss: 5.422158574219793e-05
Epoch 4857, Loss: 0.0025757667717698496, Final Batch Loss: 3.544432547641918e-05
Epoch 4858, Loss: 0.0005081676268900992, Final Batch Loss: 1.7727276144796633e-06
Epoch 4859, Loss: 0.0008765353450144175, Final Batch Loss: 5.552734000957571e-05
Epoch 4860, Loss: 0.012802461102637608, Final Batch Loss: 6.158063570183003e-06
Epoch 4861, Loss: 0.0003038084651052486, Final Batch Loss: 7.540686056017876e-05
Epoch 4862, Loss: 0.0003865583385049831, Final Batch Loss: 0.0001513498864369467
Epoch 4863, Loss: 0.005661087574480916, Final Batch Loss: 0.005254814401268959
Epoch 4864, Loss: 0.0007890068518463522, Final Batch Loss: 3.524182830005884e-06
Epoch 4865, Loss: 9.223125925927889e-05, Final Batch Loss: 1.4956456652726047e-05
Epoch 4866, Loss: 0.00012334419625403825, Final Batch Loss: 3.430384094826877e-05
Epoch 4867, Loss: 5.574834767685388e-05, Final Batch Loss: 6.431026577047305e-06
Epoch 4868, Loss: 0.00038480

Epoch 4960, Loss: 0.0055983132115215994, Final Batch Loss: 0.00533036794513464
Epoch 4961, Loss: 0.0038635080763924634, Final Batch Loss: 1.5993389752111398e-05
Epoch 4962, Loss: 0.0002254892278870102, Final Batch Loss: 5.8315392379881814e-05
Epoch 4963, Loss: 0.000147668004501611, Final Batch Loss: 1.6057980246841908e-05
Epoch 4964, Loss: 0.0002977113581437152, Final Batch Loss: 3.656770786619745e-05
Epoch 4965, Loss: 0.009331422952527646, Final Batch Loss: 7.417795131914318e-05
Epoch 4966, Loss: 0.0003868610365316272, Final Batch Loss: 0.000220410933252424
Epoch 4967, Loss: 0.0012191456771688536, Final Batch Loss: 3.928559453925118e-05
Epoch 4968, Loss: 0.0023845407649787376, Final Batch Loss: 0.0021098593715578318
Epoch 4969, Loss: 0.0012921376765007153, Final Batch Loss: 4.878109029959887e-05
Epoch 4970, Loss: 0.000511285272295936, Final Batch Loss: 1.741700543789193e-05
Epoch 4971, Loss: 0.0010778749929158948, Final Batch Loss: 9.150857658823952e-05
Epoch 4972, Loss: 0.00050892379

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[55  0  0]
 [ 2 48  0]
 [ 0  0 52]]
              precision    recall  f1-score   support

           0    0.96491   1.00000   0.98214        55
           1    1.00000   0.96000   0.97959        50
           2    1.00000   1.00000   1.00000        52

    accuracy                        0.98726       157
   macro avg    0.98830   0.98667   0.98724       157
weighted avg    0.98771   0.98726   0.98724       157



In [11]:
gen = Generator(z_dim = 108)
load_model(gen, "3 Label 5 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=108, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=30, bias=True)
    (4): Tanh()
  )
)

In [22]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 5)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [13]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[50  0  0]
 [ 0 50  0]
 [ 0  0 57]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        50
           1    1.00000   1.00000   1.00000        50
           2    1.00000   1.00000   1.00000        57

    accuracy                        1.00000       157
   macro avg    1.00000   1.00000   1.00000       157
weighted avg    1.00000   1.00000   1.00000       157



# User Classifier

In [14]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [15]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    elif y[k] == 7:
        y[k] = 3
    else:
        y[k] = 4

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [17]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.839553952217102, Final Batch Loss: 1.6145284175872803
Epoch 2, Loss: 4.837301015853882, Final Batch Loss: 1.6172398328781128
Epoch 3, Loss: 4.832103848457336, Final Batch Loss: 1.6099274158477783
Epoch 4, Loss: 4.832404971122742, Final Batch Loss: 1.614314317703247
Epoch 5, Loss: 4.832371234893799, Final Batch Loss: 1.6175434589385986
Epoch 6, Loss: 4.819251775741577, Final Batch Loss: 1.5992883443832397
Epoch 7, Loss: 4.825602650642395, Final Batch Loss: 1.6082749366760254
Epoch 8, Loss: 4.828278422355652, Final Batch Loss: 1.628514289855957
Epoch 9, Loss: 4.817973256111145, Final Batch Loss: 1.6059249639511108
Epoch 10, Loss: 4.798864006996155, Final Batch Loss: 1.5827860832214355
Epoch 11, Loss: 4.808717608451843, Final Batch Loss: 1.6040841341018677
Epoch 12, Loss: 4.804565191268921, Final Batch Loss: 1.6033741235733032
Epoch 13, Loss: 4.799977779388428, Final Batch Loss: 1.590522050857544
Epoch 14, Loss: 4.794031023979187, Final Batch Loss: 1.5898159742355347
Epoc

Epoch 122, Loss: 2.857576608657837, Final Batch Loss: 1.0055596828460693
Epoch 123, Loss: 2.638202488422394, Final Batch Loss: 0.8568491339683533
Epoch 124, Loss: 2.6889217495918274, Final Batch Loss: 0.8775727152824402
Epoch 125, Loss: 2.697430670261383, Final Batch Loss: 0.9317885637283325
Epoch 126, Loss: 2.613822102546692, Final Batch Loss: 0.8672077655792236
Epoch 127, Loss: 2.584209442138672, Final Batch Loss: 0.8406413793563843
Epoch 128, Loss: 2.600424110889435, Final Batch Loss: 0.8953009247779846
Epoch 129, Loss: 2.64409601688385, Final Batch Loss: 0.8530316948890686
Epoch 130, Loss: 2.528627634048462, Final Batch Loss: 0.747594952583313
Epoch 131, Loss: 2.6002153754234314, Final Batch Loss: 0.8706572651863098
Epoch 132, Loss: 2.5497177243232727, Final Batch Loss: 0.866150438785553
Epoch 133, Loss: 2.604208767414093, Final Batch Loss: 0.883480966091156
Epoch 134, Loss: 2.439949929714203, Final Batch Loss: 0.8046993017196655
Epoch 135, Loss: 2.5439692735671997, Final Batch Los

Epoch 244, Loss: 2.246470332145691, Final Batch Loss: 0.7294342517852783
Epoch 245, Loss: 2.189693331718445, Final Batch Loss: 0.6972238421440125
Epoch 246, Loss: 2.169723331928253, Final Batch Loss: 0.7296915650367737
Epoch 247, Loss: 2.1256173253059387, Final Batch Loss: 0.7211810350418091
Epoch 248, Loss: 2.11428165435791, Final Batch Loss: 0.7470353841781616
Epoch 249, Loss: 2.1838529109954834, Final Batch Loss: 0.8089057207107544
Epoch 250, Loss: 2.0742334723472595, Final Batch Loss: 0.6606023907661438
Epoch 251, Loss: 2.2646254301071167, Final Batch Loss: 0.8225829005241394
Epoch 252, Loss: 2.216134011745453, Final Batch Loss: 0.8121317028999329
Epoch 253, Loss: 2.1799646615982056, Final Batch Loss: 0.7849014401435852
Epoch 254, Loss: 2.169919013977051, Final Batch Loss: 0.7476747035980225
Epoch 255, Loss: 2.2606475353240967, Final Batch Loss: 0.8081663846969604
Epoch 256, Loss: 2.1458941102027893, Final Batch Loss: 0.6613348126411438
Epoch 257, Loss: 2.1069856882095337, Final Ba

Epoch 356, Loss: 2.008184552192688, Final Batch Loss: 0.7065636515617371
Epoch 357, Loss: 2.0276641249656677, Final Batch Loss: 0.6842998266220093
Epoch 358, Loss: 1.9321308732032776, Final Batch Loss: 0.6797393560409546
Epoch 359, Loss: 1.9239376783370972, Final Batch Loss: 0.6693888902664185
Epoch 360, Loss: 1.9585251212120056, Final Batch Loss: 0.7179808616638184
Epoch 361, Loss: 1.913045048713684, Final Batch Loss: 0.6566598415374756
Epoch 362, Loss: 1.844868242740631, Final Batch Loss: 0.6293975710868835
Epoch 363, Loss: 1.8440181612968445, Final Batch Loss: 0.5888350009918213
Epoch 364, Loss: 1.9047794938087463, Final Batch Loss: 0.6366846561431885
Epoch 365, Loss: 1.8104463815689087, Final Batch Loss: 0.5458212494850159
Epoch 366, Loss: 1.9317023158073425, Final Batch Loss: 0.6350725889205933
Epoch 367, Loss: 1.9010189175605774, Final Batch Loss: 0.6182428598403931
Epoch 368, Loss: 1.913204550743103, Final Batch Loss: 0.5714969635009766
Epoch 369, Loss: 1.919134497642517, Final 

Epoch 468, Loss: 1.7182719111442566, Final Batch Loss: 0.5587525367736816
Epoch 469, Loss: 1.6893838047981262, Final Batch Loss: 0.5710378289222717
Epoch 470, Loss: 1.6810516119003296, Final Batch Loss: 0.6180922389030457
Epoch 471, Loss: 1.6723129749298096, Final Batch Loss: 0.5546655058860779
Epoch 472, Loss: 1.6261698007583618, Final Batch Loss: 0.5382681488990784
Epoch 473, Loss: 1.7262502312660217, Final Batch Loss: 0.5835670232772827
Epoch 474, Loss: 1.7777738571166992, Final Batch Loss: 0.6432754993438721
Epoch 475, Loss: 1.582400619983673, Final Batch Loss: 0.4735668897628784
Epoch 476, Loss: 1.6934038400650024, Final Batch Loss: 0.5529405474662781
Epoch 477, Loss: 1.6804919838905334, Final Batch Loss: 0.6366568803787231
Epoch 478, Loss: 1.6948765516281128, Final Batch Loss: 0.5348029136657715
Epoch 479, Loss: 1.7063970565795898, Final Batch Loss: 0.5912700891494751
Epoch 480, Loss: 1.6820008754730225, Final Batch Loss: 0.60968416929245
Epoch 481, Loss: 1.8350696563720703, Fina

...

Epoch 4542, Loss: 0.45241865515708923, Final Batch Loss: 0.1518816500902176
Epoch 4543, Loss: 0.38066455721855164, Final Batch Loss: 0.13852666318416595
Epoch 4544, Loss: 0.5884749889373779, Final Batch Loss: 0.19052615761756897
Epoch 4545, Loss: 0.5750162899494171, Final Batch Loss: 0.16103099286556244
Epoch 4546, Loss: 0.5571713745594025, Final Batch Loss: 0.23352715373039246
Epoch 4547, Loss: 0.38370151817798615, Final Batch Loss: 0.14643335342407227
Epoch 4548, Loss: 0.4633272588253021, Final Batch Loss: 0.10544413328170776
Epoch 4549, Loss: 0.43004441261291504, Final Batch Loss: 0.13561396300792694
Epoch 4550, Loss: 0.5109570771455765, Final Batch Loss: 0.1924286037683487
Epoch 4551, Loss: 0.3622833490371704, Final Batch Loss: 0.08951309323310852
Epoch 4552, Loss: 0.3091781958937645, Final Batch Loss: 0.07800950109958649
Epoch 4553, Loss: 0.419867642223835, Final Batch Loss: 0.16408900916576385
Epoch 4554, Loss: 0.3900865763425827, Final Batch Loss: 0.13177312910556793
Epoch 4555,

Epoch 4653, Loss: 0.4128994941711426, Final Batch Loss: 0.16764025390148163
Epoch 4654, Loss: 0.3549548350274563, Final Batch Loss: 0.05916130170226097
Epoch 4655, Loss: 0.3787071853876114, Final Batch Loss: 0.08538801968097687
Epoch 4656, Loss: 0.3651565760374069, Final Batch Loss: 0.10919982194900513
Epoch 4657, Loss: 0.32471849769353867, Final Batch Loss: 0.11516063660383224
Epoch 4658, Loss: 0.40638355910778046, Final Batch Loss: 0.1532505303621292
Epoch 4659, Loss: 0.37762535363435745, Final Batch Loss: 0.07854873687028885
Epoch 4660, Loss: 0.42791692167520523, Final Batch Loss: 0.10250798612833023
Epoch 4661, Loss: 0.45061735808849335, Final Batch Loss: 0.19254842400550842
Epoch 4662, Loss: 0.39273542165756226, Final Batch Loss: 0.14357039332389832
Epoch 4663, Loss: 0.5140551030635834, Final Batch Loss: 0.1811273694038391
Epoch 4664, Loss: 0.44445547461509705, Final Batch Loss: 0.15729981660842896
Epoch 4665, Loss: 0.36087216436862946, Final Batch Loss: 0.14463375508785248
Epoch 

Epoch 4762, Loss: 0.5131600797176361, Final Batch Loss: 0.20198342204093933
Epoch 4763, Loss: 0.5183060169219971, Final Batch Loss: 0.20891313254833221
Epoch 4764, Loss: 0.4929462745785713, Final Batch Loss: 0.15886765718460083
Epoch 4765, Loss: 0.3718007206916809, Final Batch Loss: 0.1531090885400772
Epoch 4766, Loss: 0.5280414298176765, Final Batch Loss: 0.23249667882919312
Epoch 4767, Loss: 0.44564400613307953, Final Batch Loss: 0.12258648872375488
Epoch 4768, Loss: 0.40696682780981064, Final Batch Loss: 0.12952598929405212
Epoch 4769, Loss: 0.4709276258945465, Final Batch Loss: 0.1584918051958084
Epoch 4770, Loss: 0.4476172626018524, Final Batch Loss: 0.18663261830806732
Epoch 4771, Loss: 0.5181242078542709, Final Batch Loss: 0.18330733478069305
Epoch 4772, Loss: 0.5002754032611847, Final Batch Loss: 0.15994450449943542
Epoch 4773, Loss: 0.38520700484514236, Final Batch Loss: 0.09706566482782364
Epoch 4774, Loss: 0.4179753065109253, Final Batch Loss: 0.1384221464395523
Epoch 4775, 

Epoch 4873, Loss: 0.41228944063186646, Final Batch Loss: 0.138278067111969
Epoch 4874, Loss: 0.433736652135849, Final Batch Loss: 0.1327134668827057
Epoch 4875, Loss: 0.41243748366832733, Final Batch Loss: 0.11492429673671722
Epoch 4876, Loss: 0.5156259387731552, Final Batch Loss: 0.09854595363140106
Epoch 4877, Loss: 0.3942326530814171, Final Batch Loss: 0.14936695992946625
Epoch 4878, Loss: 0.4471952021121979, Final Batch Loss: 0.18211810290813446
Epoch 4879, Loss: 0.43675410002470016, Final Batch Loss: 0.14460568130016327
Epoch 4880, Loss: 0.46364420652389526, Final Batch Loss: 0.17876453697681427
Epoch 4881, Loss: 0.481264665722847, Final Batch Loss: 0.1537526547908783
Epoch 4882, Loss: 0.4902113825082779, Final Batch Loss: 0.24515345692634583
Epoch 4883, Loss: 0.4604165032505989, Final Batch Loss: 0.13365252315998077
Epoch 4884, Loss: 0.5980794280767441, Final Batch Loss: 0.19337718188762665
Epoch 4885, Loss: 0.5458622872829437, Final Batch Loss: 0.23410122096538544
Epoch 4886, Lo

Epoch 4983, Loss: 0.3795461878180504, Final Batch Loss: 0.11430060118436813
Epoch 4984, Loss: 0.3648841455578804, Final Batch Loss: 0.10381095111370087
Epoch 4985, Loss: 0.3622572347521782, Final Batch Loss: 0.13097268342971802
Epoch 4986, Loss: 0.4739703983068466, Final Batch Loss: 0.1941596120595932
Epoch 4987, Loss: 0.5111630111932755, Final Batch Loss: 0.16886018216609955
Epoch 4988, Loss: 0.40399497002363205, Final Batch Loss: 0.15199965238571167
Epoch 4989, Loss: 0.6158257722854614, Final Batch Loss: 0.21132346987724304
Epoch 4990, Loss: 0.4004140645265579, Final Batch Loss: 0.13969598710536957
Epoch 4991, Loss: 0.45544082671403885, Final Batch Loss: 0.14274634420871735
Epoch 4992, Loss: 0.48735038936138153, Final Batch Loss: 0.136031836271286
Epoch 4993, Loss: 0.45050664246082306, Final Batch Loss: 0.17609968781471252
Epoch 4994, Loss: 0.3694951832294464, Final Batch Loss: 0.13966180384159088
Epoch 4995, Loss: 0.4208468645811081, Final Batch Loss: 0.16304315626621246
Epoch 4996,

In [18]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[40  0  0  0  1]
 [ 0 28  0  2  0]
 [ 0  0 31  2  0]
 [ 0  0  3 31  0]
 [ 0  0  0  0 19]]
              precision    recall  f1-score   support

           0    1.00000   0.97561   0.98765        41
           1    1.00000   0.93333   0.96552        30
           2    0.91176   0.93939   0.92537        33
           3    0.88571   0.91176   0.89855        34
           4    0.95000   1.00000   0.97436        19

    accuracy                        0.94904       157
   macro avg    0.94950   0.95202   0.95029       157
weighted avg    0.95065   0.94904   0.94943       157



In [23]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5))

[[38  0  0  0  0]
 [ 0 35  0  0  0]
 [ 0  0 27  0  0]
 [ 0  0  0 29  0]
 [ 0  0  6 17  5]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        38
           1    1.00000   1.00000   1.00000        35
           2    0.81818   1.00000   0.90000        27
           3    0.63043   1.00000   0.77333        29
           4    1.00000   0.17857   0.30303        28

    accuracy                        0.85350       157
   macro avg    0.88972   0.83571   0.79527       157
weighted avg    0.90047   0.85350   0.81663       157

